# Collaborative Filtering

In [92]:
import numpy as np
import pandas as pd

In [93]:
class SimilarityMetric:

    def calculateSimilarity(self, u, v, u_mean, v_mean):
        pass

In [94]:
class CosineSimilarity(SimilarityMetric):
    def calculateSimilarity(self, u, v, u_mean, v_mean):
        u = np.nan_to_num(u, nan=0)
        v = np.nan_to_num(v, nan=0)
        return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v) + 1e-9)

class PCCSimilarity(SimilarityMetric):
    def calculateSimilarity(self, u, v, u_mean, v_mean):

        u_new = u - u_mean
        v_new = v - v_mean

        u_new = np.nan_to_num(u_new, nan=0)
        v_new = np.nan_to_num(v_new, nan=0)

        return np.dot(u_new, v_new) / (np.linalg.norm(u_new) * np.linalg.norm(v_new) + 1e-9)

In [95]:
class WeightedCosineSimilarity(SimilarityMetric):

    def __init__(self, weights):
        self.weights = weights

    def calculateSimilarity(self, u, v, u_mean, v_mean):
        u = np.nan_to_num(u, nan=0)
        u_weight_root = u * np.sqrt(self.weights)
        v = np.nan_to_num(v, nan=0)
        v_weight_root = v * np.sqrt(self.weights)

        return np.dot(u_weight_root, v_weight_root) / (np.linalg.norm(u_weight_root) * np.linalg.norm(v_weight_root) + 1e-9)


In [96]:
class WeightedPCCSimilarity(SimilarityMetric):

    def __init__(self, weights):
        self.weights = weights

    def calculateSimilarity(self, u, v, u_mean, v_mean):

        u_new = u - u_mean
        v_new = v - v_mean

        u_new = np.nan_to_num(u_new, nan=0)
        u_weight_root = u_new * np.sqrt(self.weights)
        v_new = np.nan_to_num(v_new, nan=0)
        v_weight_root = v_new * np.sqrt(self.weights)

        return np.dot(u_weight_root, v_weight_root) / (np.linalg.norm(u_weight_root) * np.linalg.norm(v_weight_root) + 1e-9)
        

In [97]:
class CollaborativeFiltering:

    def __init__(self, data, metric:SimilarityMetric):
        """
        Initialize CollaborativeFiltering object with user-item rating data.
        
        Parameters:
        - data: DataFrame containing user-item ratings
        """
        self.data = data
        self.similarity_matrix = None
        self.metric = metric
        self.movies = pd.read_csv("movies.csv")
        self.movies.set_index('MovieID', inplace=True)

    def calculate_similarity_matrix(self):
        pass

    def predict_ratings(self, user_id):
        pass
    
    def getMovies(self, user_id, movie_id_list):

        recommendations = []
    
        for i in movie_id_list:

            user = pd.DataFrame(self.data.loc[user_id], index = self.data.columns, columns = ['a'])
            rating = user.loc[i]
            movie = self.movies.loc[i]
            recommendations.append([i, rating['a'], movie['Title'], movie['Genres']])

        return recommendations
    
    def getSimilarityMatrix(self):
        return self.similarity_matrix

In [98]:
class CollaborativeFilteringItemItem(CollaborativeFiltering):

    def __init__(self, data, metric:SimilarityMetric):
        super().__init__(data, metric)
        self.means = self.data.mean(axis=0)
    
    def calculate_similarity_matrix(self):
        
        # user_item_matrix = self.train_data.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0)
        
        n_movies = self.data.shape[1]
        similarity_matrix = np.zeros((n_movies, n_movies))
        for i in range(n_movies):
            for j in range(i+1):
                temp = self.metric.calculateSimilarity(self.data.iloc[:, i], self.data.iloc[:, j], self.means[self.data.columns[i]], self.means[self.data.columns[j]])
                similarity_matrix[i, j] = temp
                similarity_matrix[j, i] = temp

        self.similarity_matrix = pd.DataFrame(similarity_matrix, index=self.data.columns, columns=self.data.columns)

    def predict_ratings(self, user_id):
        """
        Predict ratings for items for a given user.

        Parameters:
        - user_id: ID of the user for whom to predict ratings

        Returns:
        - DataFrame containing predicted ratings for each item
        """

        user_ratings = self.data.loc[user_id]
        predicted_ratings = pd.DataFrame(index=self.data.columns, columns=['PredictedRating'])

        for movie_id in predicted_ratings.index:
            numerator = 0
            denominator = 0

            for other_movie_id in predicted_ratings.index:
                if (other_movie_id != movie_id):
                    similarity = self.similarity_matrix.loc[movie_id, other_movie_id]
                    other_movie_rating = user_ratings[other_movie_id]

                    if not np.isnan(other_movie_rating):
                        numerator += similarity * (other_movie_rating) 
                        denominator += abs(similarity)
            
            predicted_ratings.loc[movie_id, 'PredictedRating'] = numerator / (denominator + 1e-9)
            
        return predicted_ratings
        

In [99]:
class CollaborativeFilteringUserUser(CollaborativeFiltering):
    def __init__(self, data, metric:SimilarityMetric):
        
        super().__init__(data, metric)
        self.means = self.data.mean(axis=1)

    def calculate_similarity_matrix(self):
        
        # user_item_matrix = self.train_data.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0)

        n_users = self.data.shape[0]
        similarity_matrix = np.zeros((n_users, n_users))
        for i in range(n_users):
            for j in range(i+1):
                temp = self.metric.calculateSimilarity(self.data.iloc[i, :], self.data.iloc[j, :], self.means[self.data.index[i]], self.means[self.data.index[j]])
                similarity_matrix[i, j] = temp
                similarity_matrix[j, i] = temp
                
        self.similarity_matrix = pd.DataFrame(similarity_matrix, index=self.data.index, columns=self.data.index)

    def predict_ratings(self, user_id):
        """
        Predict ratings for items for a given user.

        Parameters:
        - user_id: ID of the user for whom to predict ratings

        Returns:
        - DataFrame containing predicted ratings for each item
        """

        predicted_ratings = pd.DataFrame(index=self.data.columns, columns=['PredictedRating'])

        for movie_id in predicted_ratings.index:
            numerator = 0
            denominator = 0

            for other_user_id in self.data.index:
                other_user_rating = self.data.loc[other_user_id, movie_id]
                similarity = self.similarity_matrix.loc[user_id , other_user_id]

                if not np.isnan(other_user_rating):
                    numerator += similarity * (other_user_rating - self.means[other_user_id])
                    denominator += abs(similarity)

            predicted_ratings.loc[movie_id, 'PredictedRating'] = numerator / (denominator + 1e-9) + self.means[user_id]
            
        return predicted_ratings


In [100]:
class WeightsProvider:
    
    def getWeightsArray(self):
        pass

In [101]:
class IDFWeightsProvider(WeightsProvider):
    
    def __init__(self, data):

        data = data.fillna(0)
        arrayM = np.full((data.shape[1],), data.shape[0])
        watched = np.count_nonzero(data, axis=0)
        weights = np.log(arrayM / (watched + 1e-9))

        self.weights = np.nan_to_num(weights, nan=0)

        print(self.weights.shape)

    def getWeightsArray(self):
        return self.weights

In [102]:
class VarianceWeightsProvider(WeightsProvider):
    
    def __init__(self, data):

        temp = data.replace(0, np.NaN)
        weights = np.array(temp.var(axis=0))
        self.weights = np.nan_to_num(weights, nan=0)

        # print(self.weights.shape)
    
    def getWeightsArray(self):
        return self.weights

In [103]:
data=pd.read_csv("EncodedCombined2.csv")

user_item_matrix = data.pivot(index='UserID', columns='MovieID', values='Rating')
user_item_matrix = user_item_matrix.loc[0:1000,0:500]

## Cosine Similarity with Item Item Filtering

In [104]:
# metric = CosineSimilarity()
# # Create CollaborativeFiltering instance
# cf = CollaborativeFilteringItemItem(user_item_matrix_new, metric)

# cf.calculate_similarity_matrix()

# cf.getSimilarityMatrix()

In [105]:
# cf.getSimilarityMatrix().to_csv("Item_Item_Cosine_collaborative_filtering_similarity_matix.csv")

In [106]:
# prediction_df = cf.predict_ratings(2)

# ids = prediction_df.sort_values(by=['PredictedRating'].tail(10)).index
# movies = cf.getMovies(2, ids)

# for movie in movies:
#     print(f"Title : {movie[2]} , MovieID : {movie[0]} , Rating : {movie[1]} , Genres : {movie[3]}")

## PCC Similarity with Item Item Filtering 

In [107]:
# metric = PCCSimilarity()
# cf = CollaborativeFilteringItemItem(user_item_matrix, metric)

# cf.calculate_similarity_matrix()
# cf.getSimilarityMatrix()

In [108]:
# cf.getSimilarityMatrix().to_csv("Item_Item_PCC_collaborative_filtering_similarity_matix.csv")

In [109]:
# prediction_df = cf.predict_ratings(2)

# ids = prediction_df.sort_values(by=['PredictedRating']).tail(10).index
# movies = cf.getMovies(2, ids)

# for movie in movies:
#     print(f"Title : {movie[2]} , MovieID : {movie[0]}, Genres : {movie[3]}")

## Cosine Similarity with User User Filtering

In [110]:
# metric = CosineSimilarity()
# # Create CollaborativeFiltering instance
# cf = CollaborativeFilteringUserUser(user_item_matrix_new, metric)

# cf.calculate_similarity_matrix()
# cf.getSimilarityMatrix()

In [111]:
# cf.getSimilarityMatrix().to_csv("User_User_cosine_collaborative_filtering_similarity_matix.csv")

In [112]:
# prediction_df = cf.predict_ratings(2)

# ids = prediction_df.sort_values(by=['PredictedRating'].tail(10)).index
# movies = cf.getMovies(2, ids)

# for movie in movies:
#     print(f"Title : {movie[2]} , MovieID : {movie[0]} , Rating : {movie[1]} , Genres : {movie[3]}")

## PCC Similarity with User User Filtering

In [113]:
metric = PCCSimilarity()
cf = CollaborativeFilteringUserUser(user_item_matrix, metric)

cf.calculate_similarity_matrix()
cf.getSimilarityMatrix()

UserID,1,2,3,4,5,6,7,8,9,10,...,990,991,992,994,995,996,997,998,999,1000
UserID,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.000000,-0.707107,-0.612372,-0.051031,0.129792,0.000000,-0.003346,0.011358,-0.052793,...,0.014362,0.000000,0.083333,0.250000,0.039694,-0.111901,-0.138013,-0.064550,-0.182514,-0.157135
2,0.000000,1.000000,-0.116893,-0.202465,-0.101232,0.002934,0.164004,0.132386,0.138732,0.042182,...,0.153006,0.000000,0.000000,-0.091840,0.000000,0.188472,0.273782,0.082995,0.235387,-0.062054
3,-0.707107,-0.116893,1.000000,0.866025,0.000000,0.000000,0.088302,-0.082811,-0.008031,0.223980,...,-0.040622,0.000000,0.000000,-0.117851,-0.037424,0.158252,0.097590,0.000000,0.172076,0.277778
4,-0.612372,-0.202465,0.866025,1.000000,0.000000,0.000000,0.152944,-0.143433,-0.013910,0.129315,...,-0.035180,0.000000,0.000000,0.000000,-0.032410,0.131846,0.169031,0.000000,0.149022,0.288675
5,-0.051031,-0.101232,0.000000,0.000000,1.000000,-0.183418,-0.086031,0.002049,0.066074,-0.024247,...,0.114335,0.000000,-0.051031,0.000000,-0.056718,-0.032961,-0.211289,-0.197642,0.020180,-0.024056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,-0.111901,0.188472,0.158252,0.131846,-0.032961,-0.086210,0.282310,-0.035860,0.011776,0.017498,...,0.074457,0.030329,0.058075,0.053826,0.053977,1.000000,0.065099,-0.099845,0.141930,0.181623
997,-0.138013,0.273782,0.097590,0.169031,-0.211289,-0.016535,0.051704,-0.045718,0.197508,0.017487,...,-0.047572,0.000000,-0.069007,0.069007,0.000000,0.065099,1.000000,0.187083,0.126546,0.130120
998,-0.064550,0.082995,0.000000,0.000000,-0.197642,0.487217,0.000000,-0.092011,0.061584,0.000000,...,0.000000,-0.101130,-0.064550,0.000000,0.000000,-0.099845,0.187083,1.000000,-0.069285,0.000000


In [114]:
cf.getSimilarityMatrix().to_csv("User_User_PCC_collaborative_filtering_similarity_matix.csv")

In [115]:
prediction_df = cf.predict_ratings(2)

ids = prediction_df.sort_values(by=['PredictedRating']).tail(10).index
movies = cf.getMovies(2, ids)

for movie in movies:
    print(f"Title : {movie[2]} , MovieID : {movie[0]} , Genres : {movie[3]}")

Title : When Night Is Falling (1995) , MovieID : 49 , Genres : Drama|Romance
Title : Braveheart (1995) , MovieID : 110 , Genres : Action|Drama|War
Title : Shawshank Redemption, The (1994) , MovieID : 318 , Genres : Drama
Title : Forrest Gump (1994) , MovieID : 356 , Genres : Comedy|Romance|War
Title : Steal Big, Steal Little (1995) , MovieID : 119 , Genres : Comedy
Title : Star Maker, The (Uomo delle stelle, L') (1995) , MovieID : 124 , Genres : Drama
Title : Unstrung Heroes (1995) , MovieID : 205 , Genres : Comedy|Drama
Title : Across the Sea of Time (1995) , MovieID : 37 , Genres : Documentary
Title : Coldblooded (1995) , MovieID : 394 , Genres : Action
Title : You So Crazy (1994) , MovieID : 411 , Genres : Comedy


## Variance Weighted PCC Similarity with User User Filtering 

In [116]:
WeightProvider = VarianceWeightsProvider(user_item_matrix)

metric = WeightedPCCSimilarity(WeightProvider.getWeightsArray())
cf = CollaborativeFilteringUserUser(user_item_matrix, metric)
cf.calculate_similarity_matrix()

cf.getSimilarityMatrix()

UserID,1,2,3,4,5,6,7,8,9,10,...,990,991,992,994,995,996,997,998,999,1000
UserID,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.000000,-0.611716,-0.555335,-0.035121,0.193355,0.000000,-0.002349,0.020115,-0.051436,...,0.009790,0.000000,0.051964,0.253641,0.034842,-0.102304,-0.113187,-0.043394,-0.145895,-0.145049
2,0.000000,1.000000,-0.121951,-0.221423,-0.118368,0.002701,0.136467,0.115166,0.087337,0.050464,...,0.110893,0.000000,0.000000,-0.123085,0.000000,0.178664,0.245165,0.093066,0.177495,-0.122925
3,-0.611716,-0.121951,1.000000,0.790580,0.000000,0.000000,0.093640,-0.076687,-0.008696,0.235045,...,-0.047087,0.000000,0.000000,-0.077046,-0.030093,0.120797,0.105554,0.000000,0.126007,0.244178
4,-0.555335,-0.221423,0.790580,1.000000,0.000000,0.000000,0.170019,-0.139237,-0.015789,0.099492,...,-0.031042,0.000000,0.000000,0.058986,-0.027319,0.104202,0.191651,0.000000,0.114393,0.275938
5,-0.035121,-0.118368,0.000000,0.000000,1.000000,-0.179421,-0.082309,-0.025324,0.052461,-0.001382,...,0.166402,0.000000,-0.027630,0.000000,-0.065199,-0.059058,-0.273524,-0.220867,0.030875,-0.024739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,-0.102304,0.178664,0.120797,0.104202,-0.059058,-0.094076,0.240910,-0.067811,-0.018898,0.000173,...,0.046590,0.026415,0.032317,0.022271,0.051355,1.000000,0.110530,-0.029639,0.106986,0.146082
997,-0.113187,0.245165,0.105554,0.191651,-0.273524,-0.015041,0.057715,-0.045390,0.104540,0.063573,...,-0.042880,0.000000,-0.041493,0.087534,0.000000,0.110530,1.000000,0.192880,0.066365,0.078379
998,-0.043394,0.093066,0.000000,0.000000,-0.220867,0.477731,0.000000,-0.108278,0.035489,0.018048,...,0.000000,-0.148527,-0.034138,0.000000,0.000000,-0.029639,0.192880,1.000000,-0.066598,0.000000


In [117]:
cf.getSimilarityMatrix().to_csv("Variance_Weighted_PCC_collaborative_filtering_similarity_matix.csv")

In [118]:
prediction_df = cf.predict_ratings(2)

ids = prediction_df.sort_values(by=['PredictedRating']).tail(10).index
movies = cf.getMovies(2, ids)

for movie in movies:
    print(f"Title : {movie[2]} , MovieID : {movie[0]}, Genres : {movie[3]}")

Title : When Night Is Falling (1995) , MovieID : 49, Genres : Drama|Romance
Title : Being Human (1993) , MovieID : 418, Genres : Drama
Title : Steal Big, Steal Little (1995) , MovieID : 119, Genres : Comedy
Title : Forrest Gump (1994) , MovieID : 356, Genres : Comedy|Romance|War
Title : Star Maker, The (Uomo delle stelle, L') (1995) , MovieID : 124, Genres : Drama
Title : Unstrung Heroes (1995) , MovieID : 205, Genres : Comedy|Drama
Title : Across the Sea of Time (1995) , MovieID : 37, Genres : Documentary
Title : Tie That Binds, The (1995) , MovieID : 200, Genres : Thriller
Title : Coldblooded (1995) , MovieID : 394, Genres : Action
Title : You So Crazy (1994) , MovieID : 411, Genres : Comedy


## IDF Weighted PCC Similarity with User User Filtering

In [119]:
WeightProvider = IDFWeightsProvider(user_item_matrix)

metric = WeightedPCCSimilarity(WeightProvider.getWeightsArray())
cf = CollaborativeFilteringUserUser(user_item_matrix, metric)

cf.calculate_similarity_matrix()
cf.getSimilarityMatrix()

(487,)


UserID,1,2,3,4,5,6,7,8,9,10,...,990,991,992,994,995,996,997,998,999,1000
UserID,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.000000,-0.522737,-0.524356,-0.055979,0.241290,0.000000,-0.002727,-0.025571,0.013309,...,0.010584,0.000000,0.091152,0.175848,0.016422,-0.020108,-0.172195,-0.079209,-0.080830,-0.066367
2,0.000000,1.000000,-0.066185,-0.132780,-0.058752,0.003282,0.177985,0.062780,0.088657,0.024618,...,0.080294,0.000000,0.000000,-0.035500,0.000000,0.141684,0.230689,0.065909,0.143261,-0.123144
3,-0.522737,-0.066185,1.000000,0.750151,0.000000,0.000000,0.053846,-0.035826,-0.004373,0.182348,...,-0.039627,0.000000,0.000000,-0.086062,-0.015766,0.085347,0.064634,0.000000,0.077597,0.161592
4,-0.524356,-0.132780,0.750151,1.000000,0.000000,0.000000,0.108025,-0.071874,-0.008774,0.071817,...,-0.019629,0.000000,0.000000,-0.003318,-0.015814,0.082426,0.129667,0.000000,0.077838,0.194001
5,-0.055979,-0.058752,0.000000,0.000000,1.000000,-0.101980,-0.047015,-0.014356,0.112957,0.015013,...,0.111069,0.000000,-0.037751,0.000000,-0.034536,-0.022527,-0.137118,-0.135358,0.007183,-0.016629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,-0.020108,0.141684,0.085347,0.082426,-0.022527,-0.043144,0.225573,-0.063840,-0.024342,-0.018931,...,0.038984,0.021732,0.046308,0.074623,0.038468,1.000000,0.031747,-0.058910,0.038260,0.099762
997,-0.172195,0.230689,0.064634,0.129667,-0.137118,-0.013392,0.028295,-0.016762,0.173204,-0.002181,...,-0.041553,0.000000,-0.068974,0.043485,0.000000,0.031747,1.000000,0.161418,0.075916,0.090837
998,-0.079209,0.065909,0.000000,0.000000,-0.135358,0.288402,0.000000,-0.008664,0.054110,-0.009989,...,0.000000,-0.119168,-0.053418,0.000000,0.000000,-0.058910,0.161418,1.000000,-0.034872,0.000000


In [120]:
cf.getSimilarityMatrix().to_csv("IDF_Weighted_PCC_collaborative_filtering_similarity_matix.csv")

In [121]:
prediction_df = cf.predict_ratings(2)

ids = prediction_df.sort_values(by=['PredictedRating']).tail(10).index
movies = cf.getMovies(2, ids)

for movie in movies:
    print(f"Title : {movie[2]} , MovieID : {movie[0]}, Genres : {movie[3]}")

Title : Shawshank Redemption, The (1994) , MovieID : 318, Genres : Drama
Title : Forrest Gump (1994) , MovieID : 356, Genres : Comedy|Romance|War
Title : Steal Big, Steal Little (1995) , MovieID : 119, Genres : Comedy
Title : Star Maker, The (Uomo delle stelle, L') (1995) , MovieID : 124, Genres : Drama
Title : Unstrung Heroes (1995) , MovieID : 205, Genres : Comedy|Drama
Title : Gumby: The Movie (1995) , MovieID : 244, Genres : Animation|Children's
Title : Across the Sea of Time (1995) , MovieID : 37, Genres : Documentary
Title : Tie That Binds, The (1995) , MovieID : 200, Genres : Thriller
Title : Coldblooded (1995) , MovieID : 394, Genres : Action
Title : You So Crazy (1994) , MovieID : 411, Genres : Comedy
